This notebook is to collect important ideas to form a strong basis in the state estimation/SLAM

## Linear transformation of a Gaussian PDF
----

Random number x, \
$ X \approx N(\bar{x}, \sigma_{x}^2)$ \
Normal distribution function/Gaussian PDF,\
$f_{X}(x) = \frac{1}{\sigma_{x}\sqrt{2\pi}} \exp[\frac{-1}{2}(\frac{(x-\bar{x})}{\sigma_{x}})^2]$\
Linear transformation of the Gaussian PDF is another PDF with mean and variance transformed.
    $ X \approx N(\bar{x}, \sigma_{x}^2)$\
    X is transformed using $Y = aX+b$\
    $ Y \approx N(a\bar{x}+b, a^2\sigma_{x}^2) $\
   Previous mean $\bar{x}$ and variance got transformed to $ \bar{y} = a\bar{x}+b$ and $a^2\sigma_{x}^2$ respectively


## Linear transformation of a Multivariate Gaussian distribution
----

Random vector, \
$ X \approx N(\bar{X}, C_{x})$ mean and covariance\
Random vector X is transformed using $Y = AX+b$\, $ Y \approx N(\bar{Y}, C_{y})$ \
Previous mean $\bar{X}$ and covariance got transformed to $ \bar{Y} = A\bar{X}+b$ and $ C_y = AC_xA^T$ respectively

## Estimation Problem
----
* Represent the current **state** to be estimated as a random variable with a probabality distribution
    * Distribution mean = Best estimate of the current state (i.e Solution on the minimum mean squared error)
    * Distribution covariance = Estimate of the uncertainity around the current state (i.e Covariance of the estmation error)
* Represent the measurements or observations as a random variable with a probabality distribution
    * Distribution mean = Measurement value
    * Distribution covariance = Estimate of the uncertainity around the measurement
* Use the **dynamic system** to estimate the changes in the **state**(Mean) and te **uncertainity**(covariance) between measurements and updates with time


## Example
----
![Estimation problem](./images/estimation_problem.png)

In the above image, estimated position of a car is represented as a random variable at time step 1 $t_1$ with mean $\mu_1$ and variance (uncertainity) $\sigma_1$. 

We know that the car moves with time and we can use the dynamic system to predict the estimate forward 

![Estimation problem_2](./images/estimation_problem_2.png)

forward prediction of the estimate at time step 2 $t_2$ is represented as a random variable with mean $\mu_2$ and variance (uncertainity) $\sigma_2$

Notice that $\sigma_2^2 > \sigma_1^2$ that is because as time goes, we get less certain with the position of the vehicle. There is also position error (Mean moved from the centre to the rear end of the car) since we've had to predict forward without any new information or constraints.

now, assume we have got a sensor (GPS) to do the position measurement $ pos_{meas} \approx N(\mu_m, \sigma_{m}^2)$ with mean $\mu_m$ and measurement uncertainity $\sigma_{m}^2$

![Estimation problem_3](./images/estimation_problem_3.png)

Now, we can combine meaurement and our predicted estimate together to get a better estimate $ pos_{est} \approx N(\mu^{'}_2, \sigma^{'2}_{2})$ with mean $\mu^{'}_2$ and measurement uncertainity $\sigma^{'2}_{2}$

![Estimation problem_4](./images/estimation_problem_4.png)

once we fuse the measurement with predicted estimate, the new uncertainity is smaller than the original predicted estimate without any information i.e. $\sigma^{'2}_{2} < \sigma_1^2$ 

## Main idea
We can represent the things that we want to estimate as a probabilistic distribution, and then we can use rules of probability to fuse in different estimates and then use Bayes Theorem to update our estimates to get a better estimate using all the available information.

## Linear Kalman Filter
----

### Main Idea

* The Kalman filter makes the assumption that all the probability distributions relating to the state and measurement can be represented as Gaussian distributions.
    * The Kalman filter works by propagating the mean and covariance of the Gaussian distributions for the estimated state through time using a linear process model. This is the prediction step or process update step where the estimates move forward in time.
    * When the measurements are available, which are linear combination of the state, the Kalman filter updates the estimate, the state (mean and covariance) based on the measurements made. and covariance of the Gaussian. This is the update step or corrections step.
* These two processes form the prediction corrections step, which is recursively run with time.
    * The estimates are propagated forward in time to the current time. If this measurement information available at the current time, it is used to improve the current state estimates. This is then repeated as time moves forward.

### Discrete Time System Model

Process model:
$x_k = F_{k-1}x_{k-1} + G_{k-1}u_{k-1} + L_{k-1}w_{k-1} $\
Measurement model: 
$z_k = H_{k}x_{k} + M_{k}v_{k}  $

$x_k$ = State Vector\
$u_k$ = Control Input Vector\
$w_k$ = Process Model Noise Vector\
$v_k$ = Measurement Noise Vector\

$F_k$ = State transition Matrix\
$G_k$ = Control Input Matrix\
$H_k$ = Measurement Matrix\
$L_k$ = Process Model Noise Sensitivity Matrix\
$M_k$ = Measurement Noise Sensitivity Matrix\

Assumptions,
* Both process noise and measurement noise is Gaussian with zero mean and given covariance matrix  
    * $ w_K \approx N(0, Q_k)$\
    * $ v_K \approx N(0, R_k)$\
* Not correlated with time
    * $E(w_k w^T_j) = Q_k \delta_{k-j}$
    * $E(v_k v^T_j) = R_k \delta_{k-j}$
* Process and Measurement noises are independent
    * $E(w_k v^T_j) = 0 $

## Main Goal
----

Goal is to estimate the state of the system with knowledge of the Dynamics and the available noisy measurements

Posteriori Estimated state $ \hat{x}^+_k $ (Inludes the measurements $z_k$),\
$ \hat{x}^+_k = E(x_k|z_1, .....z_{k-1}, z_k)$\
$x_k = $ true state which is unknown\
$(z_1, .....z_{k-1}, z_k)$ Noisy meaurements

Priori estimated state $ \hat{x}^-_k $ (Inludes the measurements until $z_{k-1}$),\
$ \hat{x}^-_k = E(x_k|z_1, .....z_{k-1}, z_k)$

with Initial state, $ \hat{x}^+_0 = E(x_0)$

The Kalman filter also uses the covariance of the **estimation state error!** to probabilistically find the updated state estimate

Priori estimated covariance $ P^-_k = E[(x_k-\hat{x}^-_k)(x_k-\hat{x}^-_k)^T]$\

Posteriori Estimated covariance $ P^+_k = E[(x_k-\hat{x}^+_k)(x_k-\hat{x}^+_k)^T]$

estimation error $\tilde{x}_k = x_k - \hat{x}_k$

## 2D Tracking Filter
----

##### Main Goal

To estimate the position and velocity of a moving vehicle based on positional measurements

##### Assumptions

1. Vehicle travcels at a constant speed
2. Motion restricted to 2D X-Y plane in any direction
3. Input accelration of the vehicle is random and can be modelled as a random variable
4. We do get a positional measurements of the vehicle (GPS)

![Estimation problem_5](./images/estimation_problem_5.png)

##### Dynamic system

Dynamic system from the equations of motion

$a = \frac{dv}{dt} = \frac{d^2p}{dt^2}$

with constant accelration, solution to ODE

$p = p_0 + v_0t + \frac{1}{2} a t^2$

$v = v_0 + a t$

##### Dynamic system in Matrix form

$$\begin{bmatrix} p \\ v \end{bmatrix} = \begin{bmatrix} 1 & t \\ 0 & 1 \end{bmatrix}  \begin{bmatrix} p_0 \\ v_0 \end{bmatrix}   + \begin{bmatrix} \frac{1}{2} t^2 \\ t \end{bmatrix} a $$


##### Dynamic system in Discrete Matrix form

$t = \Delta{t}$

$$\begin{bmatrix} p \\ v \end{bmatrix}_k = \begin{bmatrix} 1 & \Delta{t} \\ 0 & 1 \end{bmatrix}  \begin{bmatrix} p \\ v \end{bmatrix}_{k-1}   + \begin{bmatrix} \frac{1}{2} \Delta{t}^2 \\ \Delta{t} \end{bmatrix} \begin{bmatrix} a \end{bmatrix}_{k-1}$$

##### 2D Discrete process model

state vector $x_k = \begin{bmatrix} p_x \\ p_y \\ v_x \\ v_y \end{bmatrix}$ 

Input vector $u_k = \begin{bmatrix} a_x \\ a_y \end{bmatrix}$

Process model $x_k = Fx_{k-1} + Gu_{k-1}$

$$\begin{bmatrix} p_x \\ p_y \\ v_x \\ v_y \end{bmatrix}_k = \begin{bmatrix} 1 &0 &\Delta{t} &0 \\ 0 &1 &0 &\Delta{t} \\ 0 &0 &1 &0 \\ 0 &0 &0 &1  \end{bmatrix}  \begin{bmatrix} p_x \\ p_y \\ v_x \\ v_y \end{bmatrix}_{k-1}   + \begin{bmatrix} \frac{1}{2}\Delta{t}^2 &0 \\ 0  &\frac{1}{2}\Delta{t}^2  \\ \Delta{t} &0 \\ 0 &\Delta{t} \end{bmatrix} \begin{bmatrix} a_x \\ a_y \end{bmatrix}_{k-1}$$


##### 2D Discrete process model (Random accelration)

General discrete process model above assumes that the accerelation is a deterministic known value which doesn't fit the our problem definition. Hence we model acceleration as a random variable 

state vector $x_k = \begin{bmatrix} p_x \\ p_y \\ v_x \\ v_y \end{bmatrix}$ 

Noise vector $w_k = \begin{bmatrix} a_x \\ a_y \end{bmatrix}$

$a_x \approx N(0, \sigma_{a_x}^2)$

$a_y \approx N(0, \sigma_{a_y}^2)$

$$\require{cancel}$$  
Process model $x_k = Fx_{k-1} + \cancelto{0}{Gu_{k-1}}$ where $Lw_{k-1}$ modelled as Noise.

$$\begin{bmatrix} p_x \\ p_y \\ v_x \\ v_y \end{bmatrix}_k = \begin{bmatrix} 1 &0 &\Delta{t} &0 \\ 0 &1 &0 &\Delta{t} \\ 0 &0 &1 &0 \\ 0 &0 &0 &1  \end{bmatrix}  \begin{bmatrix} p_x \\ p_y \\ v_x \\ v_y \end{bmatrix}_{k-1}   + \begin{bmatrix} \frac{1}{2}\Delta{t}^2 &0 \\ 0  &\frac{1}{2}\Delta{t}^2  \\ \Delta{t} &0 \\ 0 &\Delta{t} \end{bmatrix} \begin{bmatrix} a_x \\ a_y \end{bmatrix}_{k-1}$$

We are gonna use $a_x \approx N(0, \sigma_{a_x}^2)$ and $a_y \approx N(0, \sigma_{a_y}^2)$ as tuning parameter since we cannot measure acceleration.


#### Kalman filter prediction step (state)

* Begin the estimation step with initian condition $\hat{x}^+_0$. The $\hat{x}^+_0$ is the best estimate of the inital state of the system.
* Propagate in time to find $\hat{x}^-_1 = E(x_1)$ (Use the dynamic model)
    * $\hat{x}^-_1 = F_0\hat{x}^+_0 + G_0 u_0$
    * in general,  $\hat{x}^-_k = F_{k-1}\hat{x}^+_{k-1} + G_{k-1} u_{k-1}$
    

#### Kalman filter prediction step (covariance)

* Begin with $P^+_0$, initial covariance for the state estimate of $x_0$
    * if we have the exact knowledge of the $x_0$ then, $P^+_0 = 0$
    * if we DON'T have the exact knowledge of the $x_0$ then, $P^+_0 -> \infty$
    * Usually we have an approximate value (best guess) of $x_0$. Therefore, $P^+_0 = E[(x_0 - \hat{x}^+_0)(x_0 - \hat{x}^+_0)^T]$ 
* Propagate in time using state transition matrix $P^-_1 = F_0 P^+_0 F^T_0$, which corresponds to the process model $x_k = Fx_{k-1} + Gu_{k-1}$ (Recall the section liner transformation of gaussian covariance). 
* But we do have modelled acceleration as noise and our process model is $x_k = Fx_{k-1} + \cancelto{0}{Gu_{k-1}}$. To account for thie new noise model, we have to add new uncertainity term to covariance 
    * $P^-_1 = F_0 P^+_0 F^T_0 + L_0 Q_0 L^T_0 $
    * Usually, $L_k = I$, therefore 
    * $P^-_1 = F_0 P^+_0 F^T_0 + Q_0 $
    
 
#### Kalman filter prediction step summary

* Update/shift the mean
    * $\hat{x}^-_k = F_{k-1}\hat{x}^+_{k-1} + \cancelto{0}{G_{k-1}u_{k-1}}$
* Update/shift the covariance
    * $P^-_k = F_{k-1} P^+_{k-1} F^T_{k-1} + L^T_{k-1} Q_{k-1} L^T_{k-1}$
    
#### Kalman filter prediction step implementation details

1. Assume the initial position is (X,Y) = (0,0) m
2. Assume the initial velocity is 5 m/s at 45 degrees (VX,VY) = (5*cos(45deg),5*sin(45deg)) m/s
3. Parameters
    1. initial_position_std (covariance) = 0 # std= standard Deviation $\sigma_{p_x}^2$ and $\sigma_{p_y}^2$
    2. initial_velocity_std (covariance) = 0 # $\sigma_{v_x}^2$ and $\sigma_{v_y}^2$
    3. acceleration_std (Noise) = 0 # $\sigma_{a_x}^2$ and $\sigma_{a_y}^2$
4. Initialization
    1.  Initialize the state vector,
        1.  $\begin{bmatrix} p_x \\ p_y \\ v_x \\ v_y \end{bmatrix} = \begin{bmatrix} 0.0 \\ 0.0 \\ 5*cos(45deg) \\ 5*sin(45deg)) \end{bmatrix}$
    2. Intialize covariance matrix. 
        1. $\begin{bmatrix} \sigma_{p_x}^2 &0 &0 &0 \\ 0 &\sigma_{p_y}^2 &0 &0 \\ 0 &0 &\sigma_{v_x}^2 &0  \\ 0 &0 &0 &\sigma_{v_y}^2  \end{bmatrix} = \begin{bmatrix} initial\_position\_std*initial\_position\_std &0 &0 &0 \\ 0 &initial\_position\_std*initial\_position\_std &0 &0 \\ 0 &0 &initial\_velocity\_std*initial\_velocity\_std &0 \\ 0 &0 &0 &initial\_velocity\_std*initial\_velocity\_std  \end{bmatrix}$
5. Predict step
    1. Calculate state transition matrix,
        1. $ \begin{bmatrix} F_{k-1} \end{bmatrix}= \begin{bmatrix} 1 &0 &\Delta{t} &0 \\ 0 &1 &0 &\Delta{t} \\ 0 &0 &1 &0  \\ 0 &0 &0 &1  \end{bmatrix}$
    2. Calculate $\hat{x}^-_k = F_{k-1}\hat{x}^+_{k-1}$
    3. Initiate Process Noise details,
        1. $\begin{bmatrix} Q \end{bmatrix} = \begin{bmatrix} \sigma_{a_x}^2 &0 \\ 0 &\sigma_{a_y}^2 \end{bmatrix}$
        2. $\begin{bmatrix} L \end{bmatrix} = \begin{bmatrix} \frac{1}{2}\Delta{t}^2 &0 \\ 0 &\frac{1}{2}\Delta{t}^2 \\ \frac{1}{2}\Delta{t}^2 &0 \\ 0 &\frac{1}{2}\Delta{t}^2 \end{bmatrix}$
    4. Calculate $P^-_k = F P^+_{k-1} F^T + L Q L^T$
    
#### Validation of the kalman filter prediction step

1. If you have the simulation of the system that is being estimated, then run the simulation from the **known initial condition** without any noise. Then run the kalman filter prediction from the same initial conditions see that the 2 are consistent. Run the kalman fiter without the update step to eliminate all the hidden error in the prediction step.
    1. zero initial state, zero covariance, zero noise etc.
2. Covariance must always represent the approximate level of uncertainity in the filter estimates
    1. True state is within the uncertainity bounds around the estimated state. If this is not the case, then the filter is inconsistent.
3. check if the prediction step correctly transforms the uncertainity
    1. uncertainity of the system should not be increasing (if $Q = 0$) but it can be transformed with the model $P^-_k = F P^+_{k-1} F^T + L Q L^T$
    2. The term $F P^+_{k-1} F^T$ only transforms the uncertainity from $k-1$ step to $k$
    3. Process model noise term $L Q L^T$ inflates/increases the uncertainity
4. Filter covariance prediction model check
    1. Position uncertainity check
        1. Set the initial position uncertainity to a non-zero value
        2. set the inital velocity uncertainity = 0.0
        3. Run the filter and the check if the position uncertainity (3$\sigma$) remains same
            1. initial_position_std = 5 
            2. initial_velocity_std = 0
            3. acceleration_std = 0 
    1. Velocity uncertainity check
        1. Set the initial position uncertainity to = 0.0
        2. set the inital velocity uncertainity to a non-zero value
        3. Run the filter and the check if the velocity uncertainity (3$\sigma$) remains same but the position uncertainity growns lineraly
            1. initial_position_std = 0 
            2. initial_velocity_std = 1
                1. 1 m/s (1$\sigma$) velocity uncertainity will grow to 120m (1$\sigma$) positional uncertainity in 120 sec !!! 
            3. acceleration_std = 0 
      1. Process model Noise check
        1. Set the initial position uncertainity to = 0.0
        2. set the inital velocity uncertainity to = 0.0
        3. set the inital acceleration uncertainity to a non-zero value
        3. Run the filter and the check if the total system level uncertainity (3$\sigma$) grows with time( velocity uncertainty grows lineraly and position uncertainity growns quadratically)
            1. initial_position_std = 0 
            2. initial_velocity_std = 0 
            3. acceleration_std = 0.1
            
            
#### Kalman filter update step (state)

1. Measurement Model, $z_k= H_k x_k + M_k v_k$\
2. sensor measurement is a fucntion of current state and measurement noise $N(\bar{v}, R)$ related by measurement matrix and noise sensitivty matrix respectively. In other words, measurement is a linear combination of current state corrupted by noise.
3. we need to figure out how to update the predicted state $\hat{x}^-_{k}$ with current measurement $z_k$ to form the updated state estimate $\hat{x}^+_{k}$
4. This will be done as weighted estimate based on the **error** between the measurement $z_k$ and the predicted measurement $\hat{z}_{k} = H_k \hat{x}^-_k$ 
    1. we call this **error** innovation
    2. Mean of the innovation is $\tilde{y}_k = z_k - H_k \hat{x}^-_k$
    3. covariance of the innovation is (Just transform the uncertainity in the state estimation $P_k$ and in the measurement $R_k$ to innovation, $R_k$ is how much uncertainity(covariance) in the measurement) $S_k = H_k P^-_k H^T_k + M_k R^-_k M^T_k$
        1. $z_{k} = $ measurement vector
        2. $\hat{z}_{k} = $ Predicted measurement vector
        3. $\tilde{y}_k = $ Measurement innovation (error)
        4. $S_k = $ Innovation covariance 
5. State is updated based on the size of the innovation $\tilde{y}_k$ Using the kalman gain matrix $K_k$. The kalman gain matrix $K_k$ is a ratio between innovation uncertainity and the uncertainity in the current state estimation.
    1. kalman gain matrix $K_k$ So this is basically saying if we are really certain in the current state, then we should trust that more than the measurement. However, if we're really certain in our measurement, then we should trust that one more than the state.
6. Update step is basically, 
    1. $\hat{x}^+_k = \hat{x}^-_k + K_k \tilde{y}_k$
    2. $K_k = P^-_k H^T_k S^{-1}_k$ The Kalman metrics pretty much looks at the uncertainty of the state weighted as a ratio of the uncertainty of the measurement or the innovation. So looking at this works out how much it needs to trust the measurement or how much it needs to trust.
    
#### Kalman filter update step (Covariance)

1. The whole point of the state update process is to reduce the uncertainty in the estimates and improve the accuracy so that the covariance of the estimate must change. $P^+_k  = (I - K_k H_K) P^-_k$ 

#### Kalman filter update step summary
*  $\hat{x}^+_k = \hat{x}^-_k + K_k (z_k - H_k \hat{x}^-_k) $
*  $P^+_k  = (I - K_k H_K) P^-_k$ 
*  $K_k = P^-_k H^T_k S^{-1}_k$ 
*  $S_k = H_k P^-_k H^T_k + M_k R^-_k M^T_k$

#### GPS measurement model
1.  Measurement Model, $z_k= H_k x_k + M_k v_k$
2. $\begin{bmatrix} p_x \\ p_y  \end{bmatrix}_k = \begin{bmatrix} 1 &0 &0 &0 \\ 0 &1 &0 &0 \end{bmatrix}  \begin{bmatrix} p_x \\ p_y \\ v_x \\ v_y \end{bmatrix}$
3. measurement noise $v_k = N(0, R)$
4. $\begin{bmatrix} R \end{bmatrix} = \begin{bmatrix} \sigma_{x}^2 &0 \\ 0 &\sigma_{y}^2 \end{bmatrix}$
5. $\sigma_{gps} = \sigma_{x} = \sigma_{y}$

#### Emperical facts
* The shape of the variance profile of time is pretty much controlled by two parameters. It is controlled by the Q matrix and the R Matrix. So we've seen in the past from the prediction step, the Q matrix controls the rate of increase of uncertainty inside the system. It shows you how quickly the uncertainty is going to increase with time while the R Matrix is going to control how accurate the measurement is that we fuse into the system.

* So the larger the Q matrix, the quicker the uncertainty grows between the updates, the smaller the R matrix is, the smaller is the updated covariance of the filter. So if we have a large Q metrics and a large R matrix or update, right, we might get a covariance profile that looks like Left graph. So you can see overall the uncertainty in the system is increasing, even though we're getting some updates to it. The update doesn't provide enough information to constrain the uncertainty inside the system. So this is a bad example of what we want to be. This filter isn't actually helping us all that much, because the longer we run the filter, the more uncertainty in the system we're going to get. So it's not going to be helping us to estimate anything.

![Estimation problem_6](./images/estimation_problem_6.jpeg)

* The ideal case, what we want is when we have a small Q matrix and a small R matrix or a fast update. And we'll get the uncertainty inside the system looking like this (Right image) where even though we increase the uncertainty during the predictions, whenever we get an update, it brings the uncertainty down a greater amount than increased during the prediction step.

* Over time, we can see that the uncertainty is going to be converging and it's going to converge down to a steady state value. And this steady state value is going to be based on the ratio of the Q and R Matrices, as well as the update rate.

* An interesting thing to see from the equations here is that the final covariance P is a function of Kalman game matrix K, which is a function of our covariance innovation, is now you can see during these questions here, none of these actually has to do with the current state of the system, the current state of the system X does not appear in any of these equations.

* Basically what that is saying is that our covariance with time is independent of the current state. The more updates we get, the smaller the P matrix is going to be in ideal conditions and since variance with time does not depend on the current state X, it is possible to predict how accurate filter estimates are going to be over the long term. So if we run the filter for long enough, our covariance is going to converge down to a constant non-zero value and that value is going to be based on our Q and our R matrix.

* If we know the amount of process noise in the filter and we know how good our measurements are, we can work out overall how good our state estimates are going to be. So we can use this information in the engineering design process and work out an area budget so we can work out how good our sensors have to be to make the level performance we want to.

* Depending on our application, we might find out that we don't need a really high quality sensor. We might be able to get away with using a low quality sensor at lower cost.